In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('/home/server/gli-data-science/')
import ds_db
import helper_db

import numpy as np
import gensim
from multiprocessing import  Pool

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
q = '''

select tbtdr_ponta_user, tbtdr_tbto_id, tbtdr_rating, tbtdr_review 
from tb_transaction_delivery_rating ttdr 
where tbtdr_review notnull and tbtdr_review != ''
and tbtdr_created_date between '2021-05-25' and '2021-07-26'
'''

con, cur = ds_db.connect_prd_order_3()
df = pd.read_sql(q, con)


con.close()

You are connected


In [3]:
df = df[df['tbtdr_rating'].isin([1,2])]
df = df.reset_index(drop=True)

In [4]:
df_det = helper_db.find_member_id(df, 'tbtdr_ponta_user')

You are connected to


In [5]:
df_det.head()

,PMP_MEMBER_ID,PMP_MEMBER_UNIQUE_ID,PMP_NAME,PMP_MOBILE_PHONE
0,9990013350229355,56423747,SAHILIL WIZAR,082176838692
1,9990010138638665,3125288,Gitta Jesslyn Permana,087822848181
2,9990013149303785,54385444,DEEVA,085157063327
3,9990013226669030,55063890,AGUNG A RAHMAN,0817712220
4,9990013357708115,56512200,JOSUA TURNIP,082122711447


In [6]:
ua = pd.read_csv('uninstall_alfagift.csv')
ua = ua[ua['ID'].notnull()]

In [7]:
df_det = pd.merge(df_det, ua, left_on='PMP_MEMBER_UNIQUE_ID', right_on='ID', how='left')

In [10]:
df_det_ua = df_det[df_det['ID'].notnull()]

In [11]:
pd.merge(df_det_ua[['PMP_MEMBER_ID']], df, left_on='PMP_MEMBER_ID',right_on='tbtdr_ponta_user', how='left')

,PMP_MEMBER_ID,tbtdr_ponta_user,tbtdr_tbto_id,tbtdr_rating,tbtdr_review
0,9990013226669030,9990013226669030,7101935,2.0,Produk rusak
1,9990013320077454,9990013320077454,6073703,1.0,Pengiriman Lama
2,9990013320077454,9990013320077454,6110470,1.0,Pengiriman Lama
3,9990013320077454,9990013320077454,6073703,1.0,Pengiriman Lama
4,9990013320077454,9990013320077454,6110470,1.0,Pengiriman Lama
5,9990012325606352,9990012325606352,7822162,1.0,Produk rusak Pengiriman Lama Pembayaran Sulit Kurir meminta tip Promo tidak berlaku Voucher tidak bisa digunakan Produk tidak sesuai dengan pesanan
6,9990011681890100,9990011681890100,5997713,1.0,Pengiriman Lama
7,9990011681890100,9990011681890100,5993370,1.0,pesan blm datang kecewa sama alfamard Gorontalo sndri
8,9990013163063326,9990013163063326,1435230,1.0,"Produk tidak sesuai dengan pesanan. Ada item yang ketinggalan dan salah produk, di hubungin pihak toko gk direspond"
9,9990013225028791,9990013225028791,7134777,1.0,Pengiriman Lama


In [ ]:
# from nlp_id.lemmatizer import Lemmatizer 
# from nlp_id.stopword import StopWord 
# from nlp_id.tokenizer import Tokenizer 

# import re
# import nltk
# import string

# def basic_clean_single(text):
#     text = re.sub(r'\d+', '', text)
#     text = text.translate(str.maketrans('', '', string.punctuation))
#     lemmatizer = Lemmatizer() 
    
#     stopword = StopWord() 
#     stopwords = stopword.get_stopword() 
    
#     tokenizer = Tokenizer() 
#     words = tokenizer.tokenize(text)
#     #return [lemmatizer.lemmatize(word) for word in words if word not in stopwords]
#     #return [lemmatizer.lemmatize(word) for word in words]
#     return words

# def basic_clean(df):
#     """
#     A simple function to clean up the data. All the words that
#     are not designated as a stop word is then lemmatized after
#     encoding and basic regex parsing are performed.
#     """
#     df['tbtdr_review_token'] = df['tbtdr_review'].apply(basic_clean_single)
#     return df


# # processed_docs = list(df_re['tbtdr_review'].apply(basic_clean))

In [ ]:
# def parallelize_dataframe(df, func, n_cores=32):
#     df_split = np.array_split(df, n_cores)
#     pool = Pool(n_cores)
#     df = pd.concat(pool.map(func, df_split))
#     pool.close()
#     pool.join()
#     return df

# processed_docs = parallelize_dataframe(df_re, basic_clean)

In [ ]:
cv = CountVectorizer(max_df=0.95, min_df=2)
dtm = cv.fit_transform(df['tbtdr_review'])
dtm
#shows 8839 terms and 37996 articles

In [ ]:
LDA = LatentDirichletAllocation(n_components=5,random_state=1)
LDA.fit(dtm)

In [ ]:
for index,topic in enumerate(LDA.components_):
    print(f'topic #{index} : ')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]])

In [ ]:
topic_results = LDA.transform(dtm)
df_topic_results = pd.DataFrame(topic_results, columns=[
'0_',
'1_',
'2_',
'3_',
'4_' 
])
df_result = pd.concat([df, df_topic_results], 1)
# df_output = pd.merge(df, df_result,  how='left', on=[ 'reviewId','content' ])

In [ ]:
df_result[df_result['1_'] > 0.5].head(20)